# Лекция Python 5

## Дескрипторы
### Основы

In [3]:
class Decriptor:
    def __get__(self, instance, owner):
        print(self, instance, owner, sep='\n')

class Subject:
    attr = Decriptor()

x = Subject()
x.attr

<class '__main__.Subject'>


In [7]:
class D:  # Дескриптор не данных
    def __get__(*args): print('get')
    def __set__(*args):raise AttributeError('cannot set')
class C:
    a = D()  # Атрибут a - это экземпляр дескриптора


X = C()
X.a
X.a = 99
list(X.__dict__.keys())
X.__dict__
print(C.a)

get


AttributeError: cannot set

### Пример 1

In [8]:
class Name:
    "name descriptor docs" 
    def __get__(self, instance, owner):
        print('fetch...') 
        return instance._name
    def __set__ (self, instance, value):
        print('change. . . ') 
        instance._name = value 
    def __delete__(self, instance):
        print('remove...') 
        del instance._name

class Person: 
    def __init__ (self, name):
        self._name = name 
    name = Name()

bob = Person('Bob Smith') 
print(bob.name) 
bob.name = 'Robert Smith' 
print(bob.name) 
del bob.name 
print ('-'*20) 
sue = Person('Sue Jones') 
print(sue.name) 
print(Name.__doc__)

fetch...
Bob Smith
change. . . 
fetch...
Robert Smith
remove...
--------------------
fetch...
Sue Jones
name descriptor docs


Также подобно примеру co свойством экземпляр класса дескриптора является ат­
рибутом класса и потому наследуется всеми экземплярами клиентского класса и его 
подклассов. Скажем, если мы изменим класс Person в примере следующим образом, 
то вывод сценария останется тем же самым:


In [9]:
class Super:
    def __init__(self, name) :
        self._name = name
    name = Name ()

class Person (Super) :  # Дескрипторы наследуются
#  (т.к. являются атрибутами класса)
    pass


Кроме того, когда класс дескриптора бесполезен за рамками клиентского класса, 
то вполне разумно синтаксически внедрить определение дескриптора в его клиент. 
Вот как наш пример выглядит в случае применения вложенного класса'.


In [12]:
class Person:
    def __init__ (self, name):
        self._name = name
    class Name:  # Использование вложенного класса
        "name descriptor docs"
        def __get__(self, instance, owner):
            print('fetch. . . ')
            return instance._name
        def __set__ (self, instance, value):
            print('change...')
            instance._name = value
        def __delete__(self, instance):
            print('remove...')
            del instance._name
    name = Name()

### Вычисляемые атрибуты

In [13]:
class DescSquare:
    def __init__ (self, start) :  # Каждый дескриптор имеет
    #  собственное состояние
        self.value = start
    def __get__(self, instance, owner) : # При извлечении атрибута
        return self.value ** 2
    def __set__(self, instance, value) : # При присваивании атрибута
        self .value = value  # Операция удаления и строка
    #  документации отсутствуют
class Clientl:
    X = DescSquare(3)  # Присвоить экземпляр дескриптора атрибуту класса
class Client2:
    X = DescSquare (32)  # Еще один экземпляр в другом клиентском классе
#  Можно было бы также предусмотреть
#  два экземпляра в том же самом классе
cl = Clientl()
с2 = Client2()
print(cl.X)  # 3 ** 2
cl.X = 4
print(cl.X)  # 4 ** 2
print(с2.X)  # 32 ** 2 (1024)


9
16
1024


## Использование информации состояния в дескрипторах

- Состояние дескриптора применяется для управления либо данными, используе­
мыми при внутренней работе дескриптора, либо данными, которые охватывают 
все экземпляры. Оно может варьироваться в зависимости от места появления 
атрибута (часто в зависимости от клиентского класса).
- Состояние экземпляра хранит информацию, связанную и возможно созданную 
клиентским классом. Оно может варьироваться в зависимости от экземпляра 
клиентского класса (т.е. в зависимости от объекта приложения).

In [14]:
class DescState: # Использование состояния дескриптора, (object) в Python 2.Х
    def __init__ (self, value):
        self.value = value
    def __get__ (self, instance, owner):
        print('DescState get') 
        return self.value * 10
    def __set__ (self, instance, value):
        print('DescState set') 
        self.value = value
# Клиентский класс 
class CalcAttrs:
    X = DescState(2)
    Y = 3
    def __init__(self) :
        self.Z = 4

obj = CalcAttrs()
print(obj.X, obj.Y, obj.Z)  # X вычисляется, остальные нет
obj.X = 5  # Присваивание X перехватывается
CalcAttrs.Y =6  # Y повторно присваивается в классе
obj.Z = 7  # Z присваивается в экземпляре
print(obj.X, obj.Y, obj.Z)
obj2 = CalcAttrs()  # Но X использует разделяемые данные подобно Y!
print(obj2.X, obj2.Y, obj2.Z)

DescState get
20 3 4
DescState set
DescState get
50 6 7
DescState get
50 6 4


Кроме того, для дескриптора вполне реально хранить или применять атрибут, при­
соединенный к экземпляру клиентского класса, а не к самому себе. Важно отметить, 
что в отличие от данных, хранящихся в самом дескрипторе, это делает возможными 
данные, которые способны варьироваться в зависимости от экземпляра клиентского 
класса. Дескриптор в показанном далее примере предполагает, что экземпляр имеет 
атрибут _Х, присоединенный клиентским классом, и использует его для вычисления 
значения атрибута, который он представляет:


In [16]:
class InstState: # Использование состояния экземпляра, (object) в Python 2.Х 
    def __get__ (self, instance, owner) :
        print ('InstState get') # Предполагается, что установлен клиентским классом 
        return instance._X * 10 
    def __set__ (self, instance, value):
        print('InstState set') 
        instance._X = value * 6.28
# Клиентский класс 
class CalcAttrs:
    X = InstState()  #  Атрибут класса дескриптора
    Y = 3 #  Атрибут класса
    def __init__ (self) :
        self._X = 2  #  Атрибут экземпляра
        self.Z = 4  #  Атрибут экземпляра


obj = CalcAttrs()
print(obj.X, obj.Y, obj.Z) 
obj.X = 5
CalcAttrs.Y = 6
obj. Z = 7
print(obj.X, obj.Y, obj.Z)
obj2 = CalcAttrs ()
print(obj2.X, obj2.Y, obj2.Z)

InstState get
20 3 4
InstState set
InstState get
50 6 7
InstState get
20 6 4


С состоянием дескриптора и состоянием экземпляра связаны свои роли. На самом 
деле это и есть то общее преимущество, которым дескрипторы обладают по сравне­
нию со свойствами — поскольку они имеют собственное состояние, то могут легко 
сохранять данные внутренне, не добавляя их к пространству имен в объекте экземп­
ляра клиента. В качестве сводки в следующем дескрипторе используются оба источни­
ка состояния — в self .data хранится информация для каждого атрибута, тогда как 
instance .data может изменяться от экземпляра к экземпляру клиента:


In [28]:
class DescBoth:
    def __init__(self, data):
        self.data = data
    def __get__(self, instance, owner):
        return '%s, %s' % (self.data, instance.data)
    def __set__(self, instance, value):
        instance.data = value

class Client:
    def __init__(self, data):
        self.data = data 
    managed = DescBoth('spam')

I = Client('eggs')
print(I.__dict__)

{'data': 'eggs'}


### Связь между свойствами и дескрипторами

Как упоминалось ранее, свойства и дескрипторы тесно связаны — встроенная фун­кция property является всего лишь удобным способом создания дескриптора. Теперь, 
когда вам известно, как работают оба средства, вы должны быть в состоянии видеть воз­
можность эмуляции встроенной функции property с помощью класса дескриптора:


In [29]:
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel  # Сохранить несвязанный метод
        self.__doc__ = doc  # или другие вызываемые объекты
    def __get__(self, instance, instancetype=None):
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError ("can't get attribute") # нельзя извлечь атрибут 
        return self.fget(instance)  # Передать instance экземпляру self
        # в методах доступа к свойствам 
    def __set__ (self, instance, value):
        if self.fset is None:
            raise AttributeError ("can't set attribute") # нельзя установить атрибут 
        self.fset(instance, value)
    def __delete__(self, instance):
        if self.fdel is None:
            raise AttributeError ("can't delete attribute") # нельзя удалить атрибут 
        self.fdel(instance)

class Person:
    def getName(self): print('getName...')
    def setName(self, value): print('setName...')
    name = Property(getName, setName) # Использовать подобно property()
x = Person()
x.name
x.name = 'Bob'
del x.name

getName...
setName...


AttributeError: can't delete attribute

### \_\_getattr__ и \_\_getattribute__


- Метод__ getattr__ запускается для неопределенных атрибутов — поскольку он
выполняется только для атрибутов, которые не хранятся в экземпляре или не 
наследуются от одного из его классов, используется он прямолинейно.
- Метод__ getattribute__ запускается для каждого атрибута — поскольку он
включает все, вы должны соблюдать осторожность при его применении, чтобы 
избежать рекурсивных циклов из-за передачи суперклассу операций доступа к 
атрибутам

In [ ]:
def __getattr__(self, name):
    pass # При извлечении неопределенных атрибутов [obj.паше]
def  __getattribute__(self, name):
    pass # При извлечении всех атрибутов [obj .name] 
def __setattr__(self, name, value):
    pass # При присваивании всех атрибутов [obj .name=value] 
def __delattr__(self, name):  
    pass # При удалении всех атрибутов [del obj.name]

In [30]:
class Catcher:
    def  __getattr__ (self, name): 
        print('Get: %s' % name)
    def  __setattr__ (self, name, value):
        print('Set: %s %s' % (name, value))

X = Catcher()
X.job  # Выводится Get: job
X.pay  # Выводится Get: pay
X.pay =99  

Get: job
Get: pay
Set: pay 99


Применение__ getattribute__ в этом конкретном случае работает точно так же,
но требует (только) в Python 2.Х наследования от object и обладает едва заметным 
потенциалом зацикливания:

In [31]:
class Catcher(object):  # В Python 2.Х требуется (object)
    def __getattribute__(self, name):  # Работает здесь так же, как getattr
        print('Get: %s' % name)  # Но в целом подвержен зацикливанию
# . . . остальной код остался прежним. . .

In [32]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object  # Сохранить объект
    def __getattr__(self, attrname):
        print('Trace: ' + attrname)  # Отслеживать извлечение
        return getattr(self.wrapped, attrname) # Делегировать извлечение
X = Wrapper([1, 2, 3])
X.append(4)  # Выводится Trace: append
print (X.wrapped) 

Trace: append
[1, 2, 3, 4]


### Избегание циклов в методах перехвата атрибутов


In [41]:
def __getattribute__(self, name):
    x = self.other    # ЗАЦИКЛИВАНИЕ!!!

In [ ]:
def __getattribute__(self, name): # Передача расположенному
    x = object.__getattribute__(self, 'other')             # выше суперклассу

def __setattr__(self, name, value):
    self.other = value  # Рекурсия (и возможное ЗАЦИКЛИВАНИЕ!)

def __setattr__ (self, name, value):
    self.__dict__['other'] = value  # Использование словаря атрибутов

def __setattr__ (self, name, value):
    object.__setattr__(self, 'other', value)  # Передача расположенному выше суперклассу

def __getattribute__(self, name) :
    x = self.__dict__['other']    # Зацикливание!


## Пример 2



In [35]:
class Person:  # Код переносимый: Python 2.Х или З.Х
    def  __init__ (self, name) :  # При [Person]
        self._name = name  # Запускается__setattr__ !
    def __getattr__(self, attr) :  # При [obj.неопределенный_атрибут]
        print('get: ' + attr)
        if attr == 'name' :  # Перехват имени name: не хранится в экземпляре
            return self._name  # Зацикливания нет: реальный атрибут
        else:  # Остальные являются ошибками
            raise AttributeError(attr) 
    def __setattr__(self, attr, value):
        print('set: ' + attr) 
        if attr == 'name' :
            attr = '_name' 
        self.__dict__[attr] = value
    def __delattr__(self, attr):
        print('del: ' + attr) 
        if attr == 'name' : 
            attr = '_name' 
        del self.__dict__[attr]

bob = Person('Bob Smith') 
print(bob.name) 
bob.name = 'Robert Smith' 
print(bob.name) 
del bob.name 
print('-'*20) 
sue = Person('Sue Jones') 
print(sue.name)
sue.age = 20
# print(Person.name.__doc__ )


set: _name
get: name
Bob Smith
set: name
get: name
Robert Smith
del: name
--------------------
set: _name
get: name
Sue Jones
set: age


### Использование \_\_getattribute__


In [37]:
def __getattribute__ (self, attr) :  # При [obj .любой_атрибут]
    print('get: ' + attr) 
    if attr == 'name' :  # Перехват всех имен
        attr = '_name'  # Отображение на внутреннее имя
    return object.__getattribute__(self, attr) 

In [39]:
class Person:  # Код переносимый: Python 2.Х или З.Х
    def  __init__ (self, name) :  # При [Person]
        self._name = name  # Запускается__setattr__ !
    def __getattribute__ (self, attr) :  # При [obj .любой_атрибут]
        print('get: ' + attr) 
        if attr == 'name' :  # Перехват всех имен
            attr = '_name'  # Отображение на внутреннее имя
        return object.__getattribute__(self, attr) 
    def __setattr__(self, attr, value):
        print('set: ' + attr) 
        if attr == 'name' :
            attr = '_name' 
        self.__dict__[attr] = value
    def __delattr__(self, attr):
        print('del: ' + attr) 
        if attr == 'name' : 
            attr = '_name' 
        del self.__dict__[attr]

bob = Person('Bob Smith') 
print(bob.name) 
bob.name = 'Robert Smith' 
print(bob.name) 
del bob.name 
print('-'*20) 
sue = Person('Sue Jones') 
print(sue.name) 

set: _name
get: __dict__
get: name
Bob Smith
set: name
get: __dict__
get: name
Robert Smith
del: name
get: __dict__
--------------------
set: _name
get: __dict__
get: name
Sue Jones


### Вычисляемые атрибуты


In [40]:
class AttrSquare:
    def __init__(self, start):
        self.value = start  # Запускается__setattr__ !
    def __getattr__(self, attr):  # При операциях извлечения
    # неопределенных атрибутов 
        if attr == 'X' :
            return self.value ** 2  # value не является неопределенным
        else:
            raise AttributeError(attr)
    def __setattr__(self, attr, value): # При операциях присваивания всех атрибутов
        if attr == 'X' :
            attr = 'value'
        self.__dict__[attr] = value
A = AttrSquare(3) 
B = AttrSquare(32)
#  2 экземпляра класса с перегрузкой
#  Каждый имеет отличающуюся информацию состояния
print(A.X) # 3 *★ 2
A.X = 4
print(A.X) # 4 ** 2
print(B.X) # 32 *★ 2 

9
16
1024


### Сравнение \_\_getattr__ и \_\_getattribute__

Чтобы подытожить отличия между__ getattr__ и___getattribute__ , в следу­
ющем примере оба метода используются для реализации трех атрибутов — атрибу­
та класса attrl, атрибута экземпляра attr2 и виртуального управляемого атрибута 
attr3, вычисляемого при извлечении:


In [41]:
class GetAttr:
    attrl = 1
    def __init__ (self) :
        self.attr2 = 2
    def __getattr__(self, attr):
        print('get: ' + attr)
        if attr == 'attr3': 
            return 3
        else:
            raise AttributeError(attr)

X = GetAttr()
print(X.attrl)
print(X.attr2)
print(X.attr3)
print('-'*20)

class GetAttribute(object):  # Добавить (object) в Python 2.X
    attrl = 1
    def __init__ (self) :
        self.attr2 = 2
    def __getattribute__(self, attr) : # При операциях извлечения всех атрибутов
        print('get: ' + attr)  # Использование суперкласса во избежание зацикливания
        if attr == 'attr3':
            return 3
        else:
            return object.__getattribute__(self, attr)

X = GetAttribute()
print(X.attrl)
print(X.attr2)
print(X.attr3)


1
2
get: attr3
3
--------------------
get: attrl
1
get: attr2
2
get: attr3
3


Версия c__ getattr__ перехватывает только доступ к атрибуту attr3, т.к. он не
определен. С другой стороны, версия с__ getattribute__ перехватывает операции
извлечения всех атрибутов и обязана направлять те, которыми она не управляет, ме­
тоду извлечения из суперкласса, чтобы избежать появления циклов.

Хотя метод__ getattribute___способен перехватывать больше операций извле­
чения атрибутов, чем__ getattr__ , на практике они часто являются лишь вариация­
ми на тему — если атрибуты не хранятся физически, то оба метода обеспечивают тот 
же самый эффект.

### Снова о классах для регистрации и обработки сведений о людях, основанных на делегировании


In [44]:
class Person:
    def __init__ (self, name, job=None, pay=0):
        self.name = name
        self.job = job
        self.pay = pay
    def lastName(self):
        return self.name.split () [-1]
    def giveRaise(self, percent):
        self.pay = int(self.pay * (1 + percent))
    def __repr__ (self):
        return ' [Person: %s, %s] ' % (self.name, self.pay)

class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay) # Внедрение объекта Person 
    def giveRaise(self, percent, bonus=.10):
        self .person. giveRaise (percent + bonus) # Перехват и делегирование 
    def __getattr__(self, attr) :
        return getattr(self.person, attr)  # Делегирование всех
    # остальных атрибутов 
    def __repr__(self):
        return str(self.person) # Снова требуется перегрузка (в Python З.Х)
if __name__ == '__main__' :
    sue = Person('Sue Jones', job='dev', pay=100000)
    print(sue.lastName())
    sue.giveRaise(.10)
    print(sue)
    tom = Manager('Tom Jones', 50000)  # Manager.__init__
    print(tom.lastName())  # Manager.__getattr__ -> Person.lastName
    tom.giveRaise(.10)  # Manager.giveRaise -> Person.giveRaise
    print (tom)  # Manager.__repr__  -> Person.__repr__


Jones
 [Person: Sue Jones, 110000] 
** lastName
** person
Jones
** giveRaise
** person
** person
 [Person: Tom Jones, 60000] 


Однако по контрасту с этим взгляните, что происходит, когда мы выводим объект 
Manager в конце сценария: вызывается метод__ repr__ класса оболочки и делегиру­
ет выполнение работы методу__ repr__ внедренного объекта Person. Имея данный
факт в виду, посмотрим, что случится, если мы удалим метод Manager.__ repr__ :

In [ ]:
# Удаление метода __str__ в классе Manager
class Manager:
    def __init__(self, name, pay):
        self.person = Person(name, 'mgr', pay) 
    def giveRaise(self, percent, bonus=.10):
        self .person.giveRaise(percent + bonus) 
    def __getattr__(self, attr):
        return getattr(self.person, attr)

In [43]:
#  Замена __getattr__ методом ___getattribute__

class Manager(object):  # Использовать (object)
    def __init__(self, name, pay):
        (name, 'mgr', pay) # Внедрение объекта Person 
    def giveRaise(self, percent, bonus=.10):
        self.person.giveRaise(percent + bonus) # Перехват и делегирование 
    def __getattribute__(self, attr) :
        print('**', attr)
        if attr in ['person', 'giveRaise']:
            return object.__getattribute__(self, attr) # Извлечение моих атрибутов
        else:
            return getattr (self .person, attr)

Вкратце история заключается в том, что основанные на делегировании классы 
вроде Manager должны переопределять некоторые методы перегрузки операций (по­
добные __ герг__ и___str__ ) для их направления внедренным объектам в Python З.Х,
но не в Python 2.Х, если только не применяются классы нового стиля. Похоже, нам до­
ступны лишь варианты использования__ getattr__ и Python 2.Х либо избыточного
переопределения методов перегрузки операций для классов оболочек в Python З.Х.
Опять-таки задача не считается невыполнимой; многие классы оболочек могут 
спрогнозировать требуемый набор методов перегрузки операций, а инструменты и 
суперклассы способны автоматизировать часть решения задачи — на самом деле мы 
изучим необходимые кодовые схемы в следующей главе. Кроме того, не все классы 
используют методы перегрузки операций (в действительности большинство классов 
приложений обычно не должны их применять). Тем не менее, об этом важно пом­
нить при работе с моделями делегирования в Python З.Х; когда методы перегрузки 
операций являются частью интерфейса объекта, то классы оболочек обязаны приспо­
собиться к ним переносимым образом за счет локального переопределения.
